In [1]:
# A warning message from DGL will appear which seems to be related to an open issue in the DGL library, 
# it won't hurt the execution of the program, please ignore it.
import quartz

Using backend: pytorch[13:51:09] /opt/dgl/src/runtime/tensordispatch.cc:43: TensorDispatcher: dlopen failed: /home/lzk/anaconda3/envs/quantum/lib/python3.8/site-packages/dgl/tensoradapter/pytorch/libtensoradapter_pytorch_1.10.2.so: cannot open shared object file: No such file or directory



## Construct QuartzContext
The following lines are used to construct a context in quartz which contains all the generated transfers under a given gate set.

In [2]:
quartz_context = quartz.QuartzContext(gate_set=['h', 'cx', 't', 'tdg'], filename='../bfs_verified_simplified.json')

In [3]:
quartz_context.num_xfers

390

In [4]:
quartz_context.get_xfers()

In [5]:
quartz_context.get_xfer_from_id(id=0)

## QASM parser
You can construct a PyQASMParser to generate a PyDAG from a QASM file.

In [6]:
parser = quartz.PyQASMParser(context=quartz_context)

In [7]:
my_dag = parser.load_qasm(filename="../circuit/example-circuits/barenco_tof_3.qasm")
my_dag.num_qubits, my_dag.num_gates

(5, 58)

## PyGraph

You can construct a PyGraph from a PyDAG.

Below are some examples showing the APIs in PyGraph.

In [8]:
my_graph = quartz.PyGraph(context=quartz_context, dag=my_dag)

In [9]:
my_graph.num_nodes, my_graph.num_edges

(58, 77)

In [10]:
my_graph.all_edges()

[(0, 1, 0, 1),
 (1, 2, 1, 0),
 (1, 5, 0, 0),
 (2, 3, 0, 1),
 (3, 4, 1, 0),
 (3, 7, 0, 0),
 (4, 5, 0, 1),
 (5, 6, 1, 0),
 (5, 8, 0, 1),
 (6, 7, 0, 1),
 (7, 8, 0, 0),
 (7, 9, 1, 0),
 (8, 10, 1, 0),
 (8, 11, 0, 0),
 (10, 11, 0, 1),
 (11, 12, 0, 0),
 (11, 13, 1, 0),
 (12, 31, 0, 0),
 (13, 14, 0, 0),
 (9, 27, 0, 1),
 (14, 15, 0, 1),
 (15, 16, 1, 0),
 (15, 19, 0, 0),
 (16, 17, 0, 1),
 (17, 18, 1, 0),
 (17, 21, 0, 0),
 (18, 19, 0, 1),
 (19, 20, 1, 0),
 (19, 22, 0, 1),
 (20, 21, 0, 1),
 (21, 22, 0, 0),
 (21, 23, 1, 0),
 (22, 24, 1, 0),
 (22, 26, 0, 0),
 (24, 26, 0, 1),
 (26, 28, 0, 0),
 (26, 29, 1, 0),
 (28, 46, 0, 0),
 (29, 44, 0, 0),
 (23, 25, 0, 0),
 (25, 27, 0, 0),
 (27, 30, 1, 0),
 (27, 33, 0, 0),
 (30, 31, 0, 1),
 (31, 32, 1, 0),
 (31, 35, 0, 0),
 (32, 33, 0, 1),
 (33, 34, 1, 0),
 (33, 36, 0, 1),
 (34, 35, 0, 1),
 (35, 36, 0, 0),
 (35, 37, 1, 0),
 (36, 38, 1, 0),
 (36, 40, 0, 0),
 (38, 40, 0, 1),
 (40, 41, 0, 0),
 (40, 42, 1, 0),
 (42, 43, 0, 0),
 (37, 39, 0, 0),
 (43, 44, 0, 1),
 (44, 4

In [11]:
my_graph.all_nodes()

In [12]:
my_graph.all_nodes_with_id()

[{'id': 0, 'node': <quartz.core.PyNode at 0x7efe005481d0>},
 {'id': 1, 'node': <quartz.core.PyNode at 0x7efe00548210>},
 {'id': 2, 'node': <quartz.core.PyNode at 0x7efe00548190>},
 {'id': 3, 'node': <quartz.core.PyNode at 0x7efe005481f0>},
 {'id': 4, 'node': <quartz.core.PyNode at 0x7efe00548170>},
 {'id': 5, 'node': <quartz.core.PyNode at 0x7efe00548230>},
 {'id': 6, 'node': <quartz.core.PyNode at 0x7efe005481b0>},
 {'id': 7, 'node': <quartz.core.PyNode at 0x7efe00548270>},
 {'id': 8, 'node': <quartz.core.PyNode at 0x7efe00548290>},
 {'id': 9, 'node': <quartz.core.PyNode at 0x7efe005482b0>},
 {'id': 10, 'node': <quartz.core.PyNode at 0x7efe005482d0>},
 {'id': 11, 'node': <quartz.core.PyNode at 0x7efe005482f0>},
 {'id': 12, 'node': <quartz.core.PyNode at 0x7efe00548310>},
 {'id': 13, 'node': <quartz.core.PyNode at 0x7efe00548330>},
 {'id': 14, 'node': <quartz.core.PyNode at 0x7efe00548350>},
 {'id': 15, 'node': <quartz.core.PyNode at 0x7efe00548370>},
 {'id': 16, 'node': <quartz.core.P

In [13]:
my_graph.get_node_from_id(id=0)

In [14]:
my_graph_dgl = my_graph.to_dgl_graph()
# my_graph.all_edges()

In [15]:
my_graph_dgl.num_edges()

77

In [16]:
all_nodes = my_graph.all_nodes()
all_nodes

The codes below shows the usage of the two APIs `PyGraph.available_xfers` and `PyGraph.apply_xfer`.

In [17]:
for node in all_nodes:
    print(my_graph.available_xfers(quartz_context, node, "int"))

[128]
[190]
[138]
[]
[208, 314]
[190]
[138]
[]
[41, 172, 173, 190]
[]
[138]
[133, 143]
[132, 142]
[]
[128]
[190]
[138]
[]
[208, 314]
[190]
[138]
[]
[41, 172, 173, 190]
[]
[138]
[364]
[133, 143]
[]
[132, 142]
[132, 142]
[208, 314]
[190]
[138]
[]
[208, 314]
[190]
[157, 164, 165]
[]
[208, 314]
[]
[151, 190, 199]
[]
[]
[128]
[]
[208, 314]
[190]
[138]
[]
[208, 314]
[190]
[157, 164, 165]
[]
[208, 314]
[]
[151, 190, 199]
[]
[]


In [18]:
new_graph = my_graph.apply_xfer(xfer=quartz_context.get_xfer_from_id(id=190), node=all_nodes[1])

In [19]:
new_graph.hash()

17470542277772598492

In [20]:
new_graph.gate_count

60

The codes below is a back tracking search implemented with the quartz python APIs.

In [22]:
# Optimizing with BFS
import heapq

candidate_hq = []
heapq.heappush(candidate_hq, my_graph)
hash_set = set()
hash_set.add(my_graph.hash())
best_graph = my_graph
best_gate_cnt = my_graph.gate_count

budget = 10_000

while candidate_hq != [] and budget >= 0:
    first_candidate = heapq.heappop(candidate_hq)
    all_nodes = first_candidate.all_nodes()
    for node in all_nodes:
        appliable_xfers = first_candidate.available_xfers(quartz_context, node, "int")
        for xfer in appliable_xfers:
            new_graph = first_candidate.apply_xfer(xfer=quartz_context.get_xfer_from_id(id=xfer), node=node)
            if new_graph.hash() not in hash_set:
                hash_set.add(new_graph.hash())
                heapq.heappush(candidate_hq, new_graph)
                if new_graph < best_graph:
                    best_graph = new_graph
                    best_gate_cnt = new_graph.gate_count
                budget -= 1
                if budget % 1_000 == 0:
                    print(f'{budget}: minimum gate count is {best_gate_cnt}')

9000: minimum gate count is 58
8000: minimum gate count is 58
7000: minimum gate count is 58
6000: minimum gate count is 58
5000: minimum gate count is 58
4000: minimum gate count is 58
3000: minimum gate count is 58
2000: minimum gate count is 58
1000: minimum gate count is 56
0: minimum gate count is 56
